In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import cv2
import numpy as np
import threading

class YOLOFaceDetectionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("SnapDetect")
        self.root.geometry("1000x700")
        self.root.configure(bg='#FFB6C1')

        self.weights_path = ""
        self.cfg_path = ""
        self.names_path = ""
      
      
       # Title
        self.title_label = tk.Label(root, text="SnapDetect", font=("Helvetica", 32, "italic"), bg='#FFB6C1', fg='black')
        self.title_label.pack(pady=10)
        
        # GUI elements
        self.panel = tk.Label(root)
        self.panel.pack(padx=10, pady=10, fill=tk.BOTH, expand=True)

        top_frame = tk.Frame(root, bg="#EAEAEA")
        top_frame.pack(fill=tk.X, pady=10, side=tk.TOP)
        

        btn_select_weights = tk.Button(top_frame, text="Select Weights", command=self.select_weights, bg="#4CAF50", fg="white")
        btn_select_weights.pack(side=tk.LEFT, padx=10)

        btn_select_cfg = tk.Button(top_frame, text="Select CFG", command=self.select_cfg, bg="#2196F3", fg="white")
        btn_select_cfg.pack(side=tk.LEFT, padx=10)

        btn_select_names = tk.Button(top_frame, text="Select Names", command=self.select_names, bg="#FFC107", fg="black")
        btn_select_names.pack(side=tk.LEFT, padx=10)

        middle_frame = tk.Frame(root, bg="#EAEAEA")
        middle_frame.pack(fill=tk.X, pady=10, side=tk.TOP)

        btn_select_image = tk.Button(middle_frame, text="Select Image", command=self.select_image, bg="#FF5722", fg="white")
        btn_select_image.pack(side=tk.LEFT, padx=10)

        btn_select_video = tk.Button(middle_frame, text="Select Video", command=self.select_video, bg="#673AB7", fg="white")
        btn_select_video.pack(side=tk.LEFT, padx=10)

        btn_live_video = tk.Button(middle_frame, text="Live Video", command=self.toggle_live_video, bg="#9C27B0", fg="white")
        btn_live_video.pack(side=tk.LEFT, padx=10)

        btn_detect_objects = tk.Button(middle_frame, text="Detect Objects", command=self.toggle_detect_objects, bg="#3F51B5", fg="white")
        btn_detect_objects.pack(side=tk.LEFT, padx=10)

        filter_frame = tk.Frame(root, bg="#EAEAEA")
        filter_frame.pack(fill=tk.X, pady=10, side=tk.TOP)

        filter_buttons = [
            ("Identity", self.toggle_identity),
            ("Box Blur", self.toggle_box_blur),
            ("Gaussian Blur", self.toggle_gaussian_blur),
            ("Median Filter", self.toggle_median_filter),
            ("Emboss Filter", self.toggle_emboss),
            ("High-Pass Filter", self.toggle_high_pass),
            ("Low-Pass Filter", self.toggle_low_pass),
            ("Gray Filter", self.toggle_gray),
            ("Edge Detection", self.toggle_edge_detection),
            ("Sharpen", self.toggle_sharpen),
            ("Sepia", self.toggle_sepia),
            ("Negative", self.toggle_negative),
            ("Cartoon", self.toggle_cartoon),
            ("Brightness", self.toggle_brightness),
            ("Contrast", self.toggle_contrast),
            ("Posterize", self.toggle_posterize),
            ("Solarize", self.toggle_solarize),
            ("Tint", self.toggle_tint)
        ]

        for text, command in filter_buttons:
            btn = tk.Button(filter_frame, text=text, command=command, bg="#61dafb", fg="#20232a")
            btn.pack(side=tk.LEFT, padx=12)

      
        self.image_path = None
        self.video_path = None
        self.image = None
        self.video_capture = None
        self.net = None
        self.classes = None
        self.output_layers = None
        self.filter_mode = None
        self.detect_objects_flag = False
        self.running = False
        self.thread = None

        #(Dimensions for images resizing)
        self.fixed_width = 416
        self.fixed_height = 416

    
    def select_weights(self):
        self.weights_path = filedialog.askopenfilename()
        self.load_yolo()

    def select_cfg(self):
        self.cfg_path = filedialog.askopenfilename()
        self.load_yolo()

    def select_names(self):
        self.names_path = filedialog.askopenfilename()
        self.load_yolo()

    def load_yolo(self):
        if self.weights_path and self.cfg_path and self.names_path:
            try:
                self.net = cv2.dnn.readNet(self.weights_path, self.cfg_path)
                self.layer_names = self.net.getLayerNames()
                self.output_layers = [self.layer_names[i - 1] for i in self.net.getUnconnectedOutLayers()]
                with open(self.names_path, "r") as f:
                    self.classes = [line.strip() for line in f.readlines()]
                messagebox.showinfo("YOLO", "YOLO model loaded successfully.")
            except Exception as e:
                messagebox.showerror("YOLO Error", f"Error loading YOLO: {e}")

    # Image and video selection methods
    def select_image(self):
        self.image_path = filedialog.askopenfilename()
        if self.image_path:
            self.load_image()

    def select_video(self):
        self.video_path = filedialog.askopenfilename()
        if self.video_path:
            if self.running:
                self.stop_running()
            else:
                self.running = True
                self.thread = threading.Thread(target=self.process_video)
                self.thread.start()

    def toggle_live_video(self):
        if self.running:
            self.stop_running()
        else:
            self.video_capture = cv2.VideoCapture(0)
            self.running = True
            self.thread = threading.Thread(target=self.process_live_video)
            self.thread.start()

    def load_image(self):
        self.image = cv2.imread(self.image_path)
        if self.image is None:
            messagebox.showerror("Image Load Error", "Failed to load image.")
            return
        self.image = cv2.cvtColor(self.image, cv2.COLOR_BGR2RGB)
        self.image = cv2.resize(self.image, (self.fixed_width, self.fixed_height))
        if self.detect_objects_flag:
            self.image = self.detect_objects(self.image)
        if self.filter_mode:
            self.image = self.apply_filter_to_frame(self.image)
        self.display_image(self.image)

    def display_image(self, image):
        image = Image.fromarray(image)
        image = ImageTk.PhotoImage(image)
        self.panel.configure(image=image)
        self.panel.image = image

    # Filter methods
    def apply_filter(self, image, kernel):
        return cv2.filter2D(image, -1, kernel)

    def apply_filter_to_frame(self, frame):
        if self.filter_mode == "Box Blur":
            kernel = np.ones((5, 5), np.float32) / 25
            return self.apply_filter(frame, kernel)
        elif self.filter_mode == "Gaussian Blur":
            return cv2.GaussianBlur(frame, (5, 5), 0)
        elif self.filter_mode == "Median Filter":
            return cv2.medianBlur(frame, 5)
        elif self.filter_mode == "Emboss":
            kernel = np.array([[0, -1, -1], [1, 0, -1], [1, 1, 0]])
            return self.apply_filter(frame, kernel)
        elif self.filter_mode == "High-Pass":
            kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
            return self.apply_filter(frame, kernel)
        elif self.filter_mode == "Low-Pass":
            kernel = np.array([[1, 1, 1], [1, 1, 1], [1, 1, 1]]) / 9
            return self.apply_filter(frame, kernel)
        elif self.filter_mode == "Gray":
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            return cv2.cvtColor(gray_frame, cv2.COLOR_GRAY2RGB)
        elif self.filter_mode == "Edge Detection":
            edges = cv2.Canny(frame, 100, 200)
            return cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)
        elif self.filter_mode == "Sharpen":
            kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
            return self.apply_filter(frame, kernel)
        elif self.filter_mode == "Sepia":
            kernel = np.array([[0.272, 0.534, 0.131],
                               [0.349, 0.686, 0.168],
                               [0.393, 0.769, 0.189]])
            return self.apply_filter(frame, kernel)
        elif self.filter_mode == "Negative":
            return cv2.bitwise_not(frame)
        elif self.filter_mode == "Cartoon":
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            gray = cv2.medianBlur(gray, 5)
            edges = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 9, 9)
            color = cv2.bilateralFilter(frame, 9, 75, 75)
            cartoon = cv2.bitwise_and(color, color, mask=edges)
            return cartoon
        elif self.filter_mode == "Brightness":
            return self.adjust_brightness(frame, 50)
        elif self.filter_mode == "Contrast":
            return self.adjust_contrast(frame, 1.5)
        elif self.filter_mode == "Posterize":
            return self.posterize(frame, 4)
        elif self.filter_mode == "Solarize":
            return self.solarize(frame)
        elif self.filter_mode == "Tint":
            return self.add_tint(frame, (0, 255, 0))  # example tint color green
        return frame
  # Filter functions
    def adjust_brightness(self, image, factor):
        return cv2.convertScaleAbs(image, alpha=1, beta=factor)

    def adjust_contrast(self, image, factor):
        return cv2.convertScaleAbs(image, alpha=factor, beta=0)

    def posterize(self, image, levels):
        return cv2.convertScaleAbs(image // (256 // levels) * (256 // levels))

    def solarize(self, image, threshold=128):
        return cv2.bitwise_not(image) if np.mean(image) > threshold else image

    def toggle_identity(self):
        self.filter_mode = None
        if self.image is not None:
            self.load_image()

    def toggle_box_blur(self):
        self.filter_mode = "Box Blur"
        if self.image is not None:
            self.load_image()

    def toggle_gaussian_blur(self):
        self.filter_mode = "Gaussian Blur"
        if self.image is not None:
            self.load_image()

    def toggle_median_filter(self):
        self.filter_mode = "Median Filter"
        if self.image is not None:
            self.load_image()

    def toggle_emboss(self):
        self.filter_mode = "Emboss"
        if self.image is not None:
            self.load_image()

    def toggle_high_pass(self):
        self.filter_mode = "High-Pass"
        if self.image is not None:
            self.load_image()

    def toggle_low_pass(self):
        self.filter_mode = "Low-Pass"
        if self.image is not None:
            self.load_image()

    def toggle_gray(self):
        self.filter_mode = "Gray"
        if self.image is not None:
            self.load_image()

    def toggle_edge_detection(self):
        self.filter_mode = "Edge Detection"
        if self.image is not None:
            self.load_image()

    def toggle_sharpen(self):
        self.filter_mode = "Sharpen"
        if self.image is not None:
            self.load_image()
            
    def toggle_negative(self):
        self.filter_mode = "Negative"
        if self.image is not None:
            self.load_image()
            
    def toggle_sepia(self):
        self.filter_mode = "Sepia"
        if self.image is not None:
            self.load_image()
    def toggle_cartoon(self):
        self.filter_mode = "Cartoon"
        if self.image is not None:
            self.load_image()

    def toggle_brightness(self):
        self.filter_mode = "Brightness"
        self.apply_filter_to_frame(self.image)

    def toggle_contrast(self):
        self.filter_mode = "Contrast"
        self.apply_filter_to_frame(self.image)

    def toggle_posterize(self):
        self.filter_mode = "Posterize"
        self.apply_filter_to_frame(self.image)

    def toggle_solarize(self):
        self.filter_mode = "Solarize"
        self.apply_filter_to_frame(self.image)

    def toggle_vignette(self):
        self.filter_mode = "Vignette"
        self.apply_filter_to_frame(self.image)

    def toggle_tint(self):
        self.filter_mode = "Tint"
        self.apply_filter_to_frame(self.image)
       
    def toggle_detect_objects(self):
        self.detect_objects_flag = not self.detect_objects_flag
        if self.image is not None:
            self.load_image()

    def stop_running(self):
        if self.running:
            self.running = False
            if self.thread and self.thread.is_alive():
                self.thread.join()
            if self.video_capture:
                self.video_capture.release()
            cv2.destroyAllWindows()
        self.thread = None

    def process_video(self):
        self.video_capture = cv2.VideoCapture(self.video_path)
        while self.running and self.video_capture.isOpened():
            ret, frame = self.video_capture.read()
            if not ret:
                break
            frame = cv2.resize(frame, (self.fixed_width, self.fixed_height))
            if self.detect_objects_flag:
                frame = self.detect_objects(frame)
            if self.filter_mode:
                frame = self.apply_filter_to_frame(frame)
            self.display_image(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        self.video_capture.release()
        self.stop_running()

    def process_live_video(self):
        while self.running and self.video_capture.isOpened():
            ret, frame = self.video_capture.read()
            if not ret:
                break
            frame = cv2.resize(frame, (self.fixed_width, self.fixed_height))
            if self.detect_objects_flag:
                frame = self.detect_objects(frame)
            if self.filter_mode:
                frame = self.apply_filter_to_frame(frame)
            self.display_image(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        self.video_capture.release()
        self.stop_running()

    def detect_objects(self, frame):
        height, width, channels = frame.shape
        blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
        self.net.setInput(blob)
        outs = self.net.forward(self.output_layers)

        class_ids = []
        confidences = []
        boxes = []

        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5:
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)

                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)

                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

        for i in indices:
            x, y, w, h = boxes[i]
            label = str(self.classes[class_ids[i]])
            confidence = confidences[i]
            color = (0, 255, 0)
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, f"{label} {confidence:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        return frame


    def close(self):
        self.stop_running()
        self.root.destroy()

root = tk.Tk()
app = YOLOFaceDetectionApp(root)
root.protocol("WM_DELETE_WINDOW", app.close)
root.mainloop()
